In [1]:
import numpy as np
import tensorflow as tf
from keras import layers
import tensorflow_datasets as tfds

2024-10-03 17:02:56.755881: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-03 17:02:56.760226: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-03 17:02:56.772548: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-03 17:02:56.792997: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-03 17:02:56.798842: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-03 17:02:56.813585: I tensorflow/core/platform/cpu_feature_gu

In [2]:
dataset, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

In [3]:
dataset

{'train': <_PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>,
 'test': <_PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>,
 'unsupervised': <_PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>}

In [4]:
train_dataset, test_dataset = dataset["train"], dataset["test"]

In [5]:
train_dataset.element_spec

(TensorSpec(shape=(), dtype=tf.string, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

In [6]:
BUFFER_SIZE = 25000
BATCH_SIZE = 64

In [7]:
[item for item in train_dataset.range(8)]

2024-10-03 17:03:09.017335: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


[<tf.Tensor: shape=(), dtype=int64, numpy=0>,
 <tf.Tensor: shape=(), dtype=int64, numpy=1>,
 <tf.Tensor: shape=(), dtype=int64, numpy=2>,
 <tf.Tensor: shape=(), dtype=int64, numpy=3>,
 <tf.Tensor: shape=(), dtype=int64, numpy=4>,
 <tf.Tensor: shape=(), dtype=int64, numpy=5>,
 <tf.Tensor: shape=(), dtype=int64, numpy=6>,
 <tf.Tensor: shape=(), dtype=int64, numpy=7>]

In [8]:
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

In [9]:
VOCAB_SIZE = 10000
encoder = tf.keras.layers.TextVectorization(max_tokens=VOCAB_SIZE)
encoder.adapt(train_dataset.map(lambda text, label: text))

2024-10-03 17:03:17.171987: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [12]:
train_dataset.map(lambda text, label: text)

<_MapDataset element_spec=TensorSpec(shape=(None,), dtype=tf.string, name=None)>

In [10]:
example = "Vocabulary limitation: It caps the number of unique words the model will consider, which helps manage computational complexity and memory usage. Handling rare words: Words that are not in the top 1000 most frequent will be treated as out-of-vocabulary (OOV) tokens."

In [20]:
# Check the vocabulary
vocab = encoder.get_vocabulary()
print(f"Vocabulary size: {len(vocab)}")
print("First 10 words in the vocabulary:")
print(vocab[:10])

# Check vectorization of a sample text
sample_text = "This movie was great! I really enjoyed it."
vectorized_text = encoder(tf.constant([sample_text]))
print("\nSample text:", sample_text)
print("Vectorized form:")
print(vectorized_text.numpy())

# Decode the vectorized text back to words
decoded_text = [vocab[i] for i in vectorized_text[0] if i != 0]
print("\nDecoded text:")
print(" ".join(decoded_text))

# Check out-of-vocabulary handling
oov_text = "This movie was supercalifragilisticexpialidocious!"
vectorized_oov = encoder(tf.constant([oov_text]))
print("\nOOV text:", oov_text)
print("Vectorized form:")
print(vectorized_oov.numpy())

decoded_oov = [vocab[i] for i in vectorized_oov[0] if i != 0]
print("\nDecoded OOV text:")
print(" ".join(decoded_oov))

Vocabulary size: 10000
First 10 words in the vocabulary:
['', '[UNK]', 'the', 'and', 'a', 'of', 'to', 'is', 'in', 'it']

Sample text: This movie was great! I really enjoyed it.
Vectorized form:
[[ 11  18  14  85  10  63 492   9]]

Decoded text:
this movie was great i really enjoyed it

OOV text: This movie was supercalifragilisticexpialidocious!
Vectorized form:
[[11 18 14  1]]

Decoded OOV text:
this movie was [UNK]


In [14]:
model = tf.keras.Sequential(
    [
        encoder,
        layers.Embedding(
            input_dim=len(encoder.get_vocabulary()), output_dim=64, mask_zero=True
        ),
        layers.Bidirectional(layers.SimpleRNN(64)),
        layers.Dense(64, activation="relu"),
        layers.Dense(1, activation="linear"),
    ]
)

In [22]:
sample_text = (
    "The movie was cool. The animation and the graphics "
    "were out of this world. I would recommend this movie."
)

# Check the output of the TextVectorization layer
vectorized_text = encoder(tf.constant([sample_text]))
print("Vectorized text shape:", vectorized_text.shape)
print("Vectorized text dtype:", vectorized_text.dtype)
print("Vectorized text:")
print(vectorized_text.numpy())

Vectorized text shape: (1, 19)
Vectorized text dtype: <dtype: 'int64'>
Vectorized text:
[[   2   18   14  652    2  737    3    2 2916   66   46    5   11  188
    10   59  368   11   18]]


In [17]:
model

<Sequential name=sequential, built=False>

In [15]:
print([layer.supports_masking for layer in model.layers])

[False, True, True, True, True]


In [25]:
# Check the output of each layer
for index, layer in enumerate(model.layers):
    if index == 0:
        intermediate_output = layer(np.array([sample_text]))
    else:
        intermediate_output = layer(intermediate_output)
    print(f"Output shape of {layer.__class__.__name__}: {intermediate_output.shape}")
    print(f"Output dtype: {intermediate_output.dtype}")
    print(intermediate_output)
    print("\n")

Output shape of TextVectorization: (1, 19)
Output dtype: <dtype: 'int64'>
tf.Tensor(
[[   2   18   14  652    2  737    3    2 2916   66   46    5   11  188
    10   59  368   11   18]], shape=(1, 19), dtype=int64)


Output shape of Embedding: (1, 19, 64)
Output dtype: <dtype: 'float32'>
tf.Tensor(
[[[-0.04120809  0.03969653 -0.00851754 ...  0.03240068  0.01108141
    0.00140482]
  [ 0.03155328 -0.00747095 -0.0040307  ...  0.03335286  0.0267132
    0.00785368]
  [ 0.01387639 -0.01176424  0.01484444 ...  0.01976022  0.01416189
   -0.00150902]
  ...
  [ 0.0030445   0.02384514 -0.02981331 ... -0.03997167  0.00829545
   -0.03643908]
  [-0.03338809  0.03482658  0.01629212 ... -0.0320698  -0.0141362
   -0.02827463]
  [ 0.03155328 -0.00747095 -0.0040307  ...  0.03335286  0.0267132
    0.00785368]]], shape=(1, 19, 64), dtype=float32)


Output shape of Bidirectional: (1, 128)
Output dtype: <dtype: 'float32'>
tf.Tensor(
[[-0.34509483  0.03049743 -0.04667048 -0.08121324 -0.19355151  0.06434804
  

In [28]:
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(1e-4),
    metrics=["accuracy"],
)
history = model.fit(
    train_dataset, epochs=10, validation_data=test_dataset, validation_steps=30
)


Epoch 1/10


/home/ts-wks-45/anaconda3/envs/ml/lib/python3.12/site-packages/keras/src/backend/tensorflow/nn.py:707: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


391/391 ━━━━━━━━━━━━━━━━━━━━ 108s 268ms/step - accuracy: 0.5269 - loss: 0.6911 - val_accuracy: 0.5578 - val_loss: 0.6823
Epoch 2/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 105s 268ms/step - accuracy: 0.6003 - loss: 0.6642 - val_accuracy: 0.7453 - val_loss: 0.5619
Epoch 3/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 106s 272ms/step - accuracy: 0.7872 - loss: 0.4871 - val_accuracy: 0.8625 - val_loss: 0.3424
Epoch 4/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 105s 267ms/step - accuracy: 0.8673 - loss: 0.3887 - val_accuracy: 0.8562 - val_loss: 0.3455
Epoch 5/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 113s 290ms/step - accuracy: 0.9168 - loss: 0.2270 - val_accuracy: 0.8698 - val_loss: 0.3443
Epoch 6/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 121s 308ms/step - accuracy: 0.9375 - loss: 0.1769 - val_accuracy: 0.8672 - val_loss: 0.3758
Epoch 7/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 134s 289ms/step - accuracy: 0.9468 - loss: 0.1539 - val_accuracy: 0.8672 - val_loss: 0.3880
Epoch 8/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 135s 271ms/step - accuracy: 0.9671 - loss: 0.10

2024-10-03 18:54:27.881679: W tensorflow/core/kernels/data/cache_dataset_ops.cc:913] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


391/391 ━━━━━━━━━━━━━━━━━━━━ 26s 66ms/step - accuracy: 0.8616 - loss: 0.5376
Test loss: 0.5264281630516052
Test accuracy: 0.8628799915313721


In [29]:
test_loss, test_acc = model.evaluate(test_dataset)
print(f"Test loss: {test_loss}")
print(f"Test accuracy: {test_acc}")

391/391 ━━━━━━━━━━━━━━━━━━━━ 26s 65ms/step - accuracy: 0.8616 - loss: 0.5376
Test loss: 0.5264281630516052
Test accuracy: 0.8628799915313721


In [30]:
test_dataset

<_BatchDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [34]:
model.predict(
    np.array(
        [
            "Your presentation was incredibly engaging and well-structured, showcasing your deep understanding of the topic; however, some of the slides were a bit text-heavy, which made it challenging for the audience to follow along at times."
        ]
    ),
    batch_size=1,
)

ValueError: Invalid dtype: str7392